<a href="https://colab.research.google.com/github/NVIDIA/synthda/blob/main/colab/synthda_loop_noCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, output
drive.mount('/content/drive')

Mounted at /content/drive


## Downloading and Setting up dependent repos (Optional if you have your own models you'd like to use instead )

In [ ]:
!mkdir -p /content/drive/MyDrive/autosynthda/indiv


!git clone https://github.com/NVIDIA/synthda /content/drive/MyDrive/autosynthda/indiv

In [ ]:
%cd /content/drive/MyDrive/autosynthda/indiv
!ls

In [ ]:
!pip install -r /content/drive/MyDrive/autosynthda/indiv/components/requirements.txt

In [ ]:
!pip install numpy==1.23.5
!pip install yacs
!pip install filterpy
!pip install smplx==0.1.28
!pip install trimesh==3.9.0
!pip install chumpy==0.70
!pip install dotenv

In [ ]:
# hardcoded fix for chumpy for python 3.11

import inspect

# Monkey patch getargspec for chumpy compatibility with Python ≥3.11
if not hasattr(inspect, 'getargspec'):
    from collections import namedtuple

    ArgSpec = namedtuple('ArgSpec', 'args varargs keywords defaults')

    def getargspec(func):
        sig = inspect.signature(func)
        args = []
        varargs = None
        keywords = None
        defaults = []

        for name, param in sig.parameters.items():
            if param.kind in (param.POSITIONAL_ONLY, param.POSITIONAL_OR_KEYWORD):
                args.append(name)
                if param.default is not param.empty:
                    defaults.append(param.default)
            elif param.kind == param.VAR_POSITIONAL:
                varargs = name
            elif param.kind == param.VAR_KEYWORD:
                keywords = name

        return ArgSpec(args, varargs, keywords, tuple(defaults) if defaults else None)

    inspect.getargspec = getargspec


## Download and Install Repos

In [ ]:
!git clone https://github.com/Vegetebird/StridedTransformer-Pose3D.git \
             /content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D

!git clone https://github.com/EricGuo5513/text-to-motion.git \
             /content/drive/MyDrive/autosynthda/indiv/text-to-motion

!git clone https://github.com/wangsen1312/joints2smpl.git \
             /content/drive/MyDrive/autosynthda/indiv/joints2smpl

%cd /content/drive/MyDrive/autosynthda/indiv

# Download Blender into the current directory
!wget -P . https://download.blender.org/release/Blender3.0/blender-3.0.0-linux-x64.tar.xz

# Extract in-place
!tar -xf blender-3.0.0-linux-x64.tar.xz -C .

#!git clone https://github.com/facebookresearch/SlowFast.git /content/drive/MyDrive/autosynthda/indiv

In [ ]:
env_vars = """
STRIDED_TRANSFORMER_PATH=/content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D
TEXT_TO_MOTION_PATH=/content/drive/MyDrive/autosynthda/indiv/text-to-motion
JOINTS2SMPL_PATH=/content/drive/MyDrive/autosynthda/indiv/joints2smpl
SLOWFAST_PATH=/content/drive/MyDrive/autosynthda/indiv/SlowFast
BLENDER_BIN=/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64/blender
BLENDER_ROOT=/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64
BLENDER_PATH=/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64/blender
"""

with open("/content/drive/MyDrive/autosynthda/indiv/synthda/components/.env", "w") as f:
    f.write(env_vars.strip())


In [ ]:
import os

# switch into the Drive-based repo folder
%cd /content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D

# make sure the pretrained checkpoint dir exists
!mkdir -p checkpoint/pretrained

# download the two refine/non-refine checkpoints if missing
if not os.path.exists('checkpoint/pretrained/refine_4365.pth'):
    !gdown https://drive.google.com/uc?id=1aDLu0SB9JnPYZOOzQsJMV9zEIHg2Uro7 \
            -O checkpoint/pretrained/refine_4365.pth
if not os.path.exists('checkpoint/pretrained/no_refine_4365.pth'):
    !gdown https://drive.google.com/uc?id=1l63AI9BsNovpfTAbfAkySo9X2MOWgYZH \
            -O checkpoint/pretrained/no_refine_4365.pth

# ensure the demo lib checkpoint dir exists
!mkdir -p demo/lib/checkpoint

# download YOLOv3 weights
if not os.path.exists('demo/lib/checkpoint/yolov3.weights'):
    !gdown https://drive.google.com/uc?id=1gWZl1VrlLZKBf0Pfkj4hKiFxe8sHP-1C \
            -O demo/lib/checkpoint/yolov3.weights

# download HRNet pose model
if not os.path.exists('demo/lib/checkpoint/pose_hrnet_w48_384x288.pth'):
    !gdown https://drive.google.com/uc?id=1CpyZiUIUlEjiql4rILwdBT4666S72Oq4 \
            -O demo/lib/checkpoint/pose_hrnet_w48_384x288.pth


In [ ]:
import os

# switch into the Drive-based repo folder
%cd /content/drive/MyDrive/autosynthda/indiv/text-to-motion

# make sure the checkpoints1 dir exists
!mkdir -p checkpoints1

# download the model checkpoint if missing
if not os.path.exists('checkpoints1/checkpoints'):
    !gdown https://drive.google.com/uc?id=12liZW5iyvoybXD8eOw4VanTgsMtynCuU \
            -O checkpoints1/checkpoints


In [ ]:
# 2. Switch into the text-to-motion folder on Drive
%cd /content/drive/MyDrive/autosynthda/indiv/text-to-motion

# 3. Ensure the checkpoints1 dir exists
!mkdir -p checkpoints1

# 4a. Download & unzip the first model
if not os.path.exists('checkpoints1/model.zip'):
    !gdown --id 12liZW5iyvoybXD8eOw4VanTgsMtynCuU -O checkpoints1/model.zip
!unzip -q checkpoints1/model.zip -d checkpoints1

# 4b. Download & unzip the second model
if not os.path.exists('checkpoints1/model2.zip'):
    !gdown --id 1IgrFCnxeg4olBtURUHimzS03ZI0df_6W -O checkpoints1/model2.zip
!unzip -q checkpoints1/model2.zip -d checkpoints1


## Run Each Component for Sanity Check

In [ ]:
## Note: You should replace the files with the edited versions to make it work in Colab. It can be found in the colab/ folder on the Github

In [1]:
# switch into the Drive-based StridedTransformer folder
%cd /content/drive/MyDrive/autosynthda/indiv/StridedTransformer-Pose3D

# run the visualization on your sample video (adjust the path if your video lives elsewhere)
!python demo/vis.py --video sample_video.mp4


In [ ]:
# switch into the Drive-mounted text-to-motion folder
%cd /content/drive/MyDrive/autosynthda/indiv/text-to-motion

# generate motions using your input.txt in that folder
!python gen_motion_script.py \
    --name Comp_v6_KLD01 \
    --text_file input.txt \
    --repeat_time 1


In [ ]:
import importlib
import smplx
importlib.reload(smplx)

# switch into the Drive-mounted joints2smpl folder
%cd /content/drive/MyDrive/autosynthda/indiv/joints2smpl

# list the SMPL model files in your Drive repo
!ls -lh /content/drive/MyDrive/autosynthda/indiv/joints2smpl/smpl_models/smpl/

# run the fitting script on your test_motion2.npy (make sure the .npy is in this folder or give a full path)
!python fit_seq.py --files test_motion2.npy


In [ ]:
import shutil
import os

# Source folder in your Drive-mounted joints2smpl repo
src = "/content/drive/MyDrive/autosynthda/indiv/joints2smpl/demo/demo_results/test_motion2"

# Destination folder in your Drive-mounted SynthDA repo
dst = "/content/drive/MyDrive/autosynthda/indiv/synthda/components/renders/test_motion2"

# Ensure the destination parent directory exists
os.makedirs(os.path.dirname(dst), exist_ok=True)

# If it already exists, remove the old one
if os.path.exists(dst):
    shutil.rmtree(dst)

# Copy the entire demo_results folder into SynthDA's renders directory
shutil.copytree(src, dst)
print("✅ Folder copied to expected Blender input location.")

# Verify
expected_path = dst
print("Exists:", os.path.exists(expected_path))
print("Contents:", os.listdir(expected_path) if os.path.exists(expected_path) else "❌ Path does not exist.")
!ls /content/drive/MyDrive/autosynthda/indiv/synthda/components/renders/test_motion2

In [ ]:
#!/content/blender-3.0.0-linux-x64/blender -b -P /content/synthda/components/animation_pose.py -- --name <folder_with_ply_files>

# may not work on on Colab immediately, as running blender headlessly on Colab has some challenges. However you will be able to download the .fbx as well to view the animation locally.
# For Colab, the suggested alternative is to use pyrender instead, which we use in the loop below!

# Step into the folder that contains the script AND the renders/ directory
%cd /content/drive/MyDrive/autosynthda/indiv/synthda/components

# Now launch Blender relative to this directory
!/content/drive/MyDrive/autosynthda/indiv/blender-3.0.0-linux-x64/blender \
    -b \
    -P animation_pose.py \
    -- --name test_motion2


## SynthDa Full Pipeline without CV model integrated
### (CV model training done separately and you would need to feed the acc and loss values into the loop here)

In [ ]:
# Install rendering & media deps
!pip install trimesh pyrender imageio[ffmpeg] pyglet av python-dotenv

# Mount Drive (if your code lives there)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

def generate_sample_metrics(metrics_dir: str, weights: list, acc_values: list, loss_values: list):
    """
    Generate sample accuracy and loss .txt files for different weights.

    Args:
        metrics_dir (str): The output directory where files will be saved.
        weights (list): List of weight floats (e.g., [0.4, 0.5, 0.6]).
        acc_values (list): Accuracy values corresponding to weights.
        loss_values (list): Loss values corresponding to weights.
    """
    os.makedirs(metrics_dir, exist_ok=True)

    for w, acc, loss in zip(weights, acc_values, loss_values):
        acc_file = os.path.join(metrics_dir, f"acc_w{w:.2f}.txt")
        loss_file = os.path.join(metrics_dir, f"loss_w{w:.2f}.txt")

        with open(acc_file, 'w') as af:
            af.write(f"{acc:.4f}")

        with open(loss_file, 'w') as lf:
            lf.write(f"{loss:.4f}")

    print(f"✅ Sample metrics written to: {metrics_dir}")

# Example: generate for different weights
generate_sample_metrics(
    metrics_dir='/content/drive/MyDrive/autosynthda/indiv/sample-loss',
    weights=[0.40, 0.50, 0.6],
    acc_values=[0.60, 0.75, 0.86],
    loss_values=[1.3, 0.95, 0.91]
)


## Key variables for the loop

In [ ]:
import os, sys
from pathlib import Path

# Offscreen EGL for PyRender
os.environ['PYOPENGL_PLATFORM'] = 'egl'

# Repo roots (adjust as needed)
ROOT        = Path('/content/drive/MyDrive/autosynthda/indiv')
COMP_ROOT   = ROOT / 'synthda' / 'components'
STRIDED     = ROOT / 'StridedTransformer-Pose3D'
JOINTS2SMPL = ROOT / 'joints2smpl'

# Make sure Python can import your .py files
sys.path.extend([
    str(ROOT),
    str(COMP_ROOT),
    str(STRIDED),
    str(JOINTS2SMPL),
])


In [ ]:
import shutil, subprocess, random, math
import numpy as np
from itertools import combinations
import sys
sys.path.append('/content/drive/MyDrive/autosynthda/indiv/synthda/components/optimisation')
from optimisation.optimisation_utils import (
    map_h36m_to_smpl, upsample_pose_data, compute_P_opt
)
import trimesh, pyrender, imageio
from PIL import Image


In [ ]:
%cd /content/drive/MyDrive/autosynthda/indiv/synthda/components
# in your /content/synthda/components folder
import torch
print(torch.__version__)


## Additional Function Definitions

### To make it faster, we used pyrender here instead of Blender. But if you are doing it locally, we recc Blender

In [ ]:
def render_ply_sequence(ply_folder: Path, png_out: Path,
                        width=1920, height=1080, fps=27):
    png_out.mkdir(parents=True, exist_ok=True)
    scene = pyrender.Scene()
    cam   = pyrender.PerspectiveCamera(yfov=math.radians(50.0))
    light = pyrender.DirectionalLight(color=[1,1,1], intensity=2.0)
    scene.add(cam,   pose=trimesh.transformations.translation_matrix([0,0,2]))
    scene.add(light, pose=trimesh.transformations.translation_matrix([0,0,2]))
    r = pyrender.OffscreenRenderer(width, height, point_size=1.0)

    ply_files = sorted(ply_folder.glob("*.ply"))
    for idx, ply in enumerate(ply_files):
        mesh = trimesh.load_mesh(str(ply))
        m = pyrender.Mesh.from_trimesh(mesh, smooth=False)
        # clear old mesh
        for node in list(scene.mesh_nodes):
            scene.remove_node(node)
        scene.add(m)
        color, _ = r.render(scene)
        Image.fromarray(color).save(png_out / f"frame_{idx:04d}.png")
        if idx % 10 == 0:
            print(f"Rendered {idx+1}/{len(ply_files)} frames")
    r.delete()


In [ ]:
def both_real_main(*, weight_A: float, input_dir: str, output_dir: str, num_pairs: int):
    random.seed(42)
    vids      = list(Path(input_dir).glob("*.mp4"))
    pairs     = list(combinations(vids, 2))
    selection = random.sample(pairs, min(num_pairs, len(pairs)))

    for v1_path, v2_path in selection:
        v1, v2    = v1_path.name, v2_path.name
        pair_name = v1_path.stem + "_" + v2_path.stem
        pair_dir  = Path(output_dir) / pair_name
        pair_dir.mkdir(parents=True, exist_ok=True)

        # 1) Copy & StridedTransformer visualisation
        for vn in (v1, v2):
            # keep a copy
            shutil.copy(Path(input_dir)/vn, pair_dir/vn)
            # stage for vis
            dst = STRIDED / "demo" / "video" / vn
            dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(Path(input_dir)/vn, dst)
            subprocess.run(
                ["python","demo/vis.py","--video", vn],
                cwd=STRIDED, check=True)

        # 2) Map NPZ → SMPL .npy
        npz1 = STRIDED/"demo"/"output"/v1_path.stem/"output_3D"/"output_keypoints_3d.npz"
        npz2 = STRIDED/"demo"/"output"/v2_path.stem/"output_3D"/"output_keypoints_3d.npz"
        out1 = pair_dir/"output_keypoints_3d_real1.npy"
        out2 = pair_dir/"output_keypoints_3d_real2.npy"
        np.save(out1, map_h36m_to_smpl(str(npz1)))
        np.save(out2, map_h36m_to_smpl(str(npz2)))

        # 3) Align & upsample
        p1, p2 = np.load(out1), np.load(out2)
        if p1.shape[0] < p2.shape[0]:
            np.save(out1, upsample_pose_data(str(out1), p2.shape[0]))
        else:
            np.save(out2, upsample_pose_data(str(out2), p1.shape[0]))

        # 4) Generate weighted mixtures
        var_dir = pair_dir/"all_variations"
        var_dir.mkdir(exist_ok=True, parents=True)
        for wA in [round(x,1) for x in np.linspace(0.1,0.9,9)]:
            wB   = round(1-wA,1)
            Popt = compute_P_opt(str(out1), str(out2), alpha=0.5, w_A=wA, w_B=wB)
            np.save(var_dir/f"euclidean_wA{wA}_wB{wB}.npy", Popt)

        # 5) Pick target variation
        target = var_dir/f"euclidean_wA{weight_A}_wB{round(1-weight_A,1)}.npy"
        if not target.exists():
            raise FileNotFoundError(f"Missing variation: {target}")

        # 6) joints2smpl → .ply
        dest_npy = JOINTS2SMPL/"demo"/"demo_data"/target.name
        dest_npy.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(target, dest_npy)
        subprocess.run(
            ["python","fit_seq.py","--files", dest_npy.name, "--num_smplify_iters","1"],
            cwd=JOINTS2SMPL, check=True)
        ply_src = JOINTS2SMPL/"demo"/"demo_results"/dest_npy.stem

        # 7) Render PLY → PNG
        png_folder = COMP_ROOT/"renders"/f"{pair_name}_{dest_npy.stem}"
        render_ply_sequence(ply_src, png_folder)

        # 8) Stitch PNG → MP4
        out_mp4 = Path(output_dir)/f"videos_real2_{weight_A}"/f"{pair_name}.mp4"
        out_mp4.parent.mkdir(parents=True, exist_ok=True)
        subprocess.run([
            "ffmpeg","-y",
            "-framerate","27",
            "-i", str(png_folder/"frame_%04d.png"),
            "-c:v","libx264","-pix_fmt","yuv420p",
            str(out_mp4)
        ], check=True)

        print("✅ Generated:", out_mp4)


### Generate starting videos for first comparisons

In [ ]:
# Configuration
weights = [0.4, 0.5, 0.6]
input_dir  = str(COMP_ROOT/"dataset"/"specific_data")
output_dir = str(COMP_ROOT/"dataset"/"data_manipulation")

for w in weights:
    print(f"\n Now running both_real_main with weight {w}")
    both_real_main(
        weight_A=w,
        input_dir=input_dir,
        output_dir=output_dir,
        num_pairs=1
    )


In [ ]:
# ─── TODO: Run your model on the generated videos to compute loss & FALL accuracy ───
# User to provide: loss_w{w}.txt and acc_w{w}.txt file paths for each w in [0.4,0.5,0.6]
# this would be the baseline accuracy that we would compare against


## SynthDa Loop

In [ ]:
import os

# ─── 1) Ask for the directory containing your loss files ──────────────────────
loss_dir = input(
    "Enter the path to the folder containing your loss_w{weight}.txt files:\n"
)

def evaluate_loss(weight: float) -> float:
    """
    Reads the model loss for the given weight from a text file.
    Supports 3 decimal places (e.g., loss_w0.643.txt).
    """
    fname = f"loss_w{weight:.3f}.txt"
    fpath = os.path.join(loss_dir, fname)
    if not os.path.isfile(fpath):
        fpath = input(f"File {fname} not found. Enter full path to loss file for weight {weight:.3f}:\n")
    return float(open(fpath).read().strip())


def calculate_new_weight(prev_w: float, prev_loss: float, step: float):
    """
    Uses finite differences: compares loss at prev_w±step,
    and returns the weight with the lowest loss among [low, prev, high],
    allowing full floating-point precision.
    """
    low  = max(prev_w - step, 0.0)
    high = min(prev_w + step, 1.0)

    # Keep full precision in filenames (up to 3 dp recommended, but to make it faster you can reduce to 1dp)
    loss_low  = evaluate_loss(round(low, 3))
    loss_high = evaluate_loss(round(high, 3))
    candidates = [
        (round(low,  3), loss_low),
        (round(prev_w, 3), prev_loss),
        (round(high, 3), loss_high)
    ]
    best_w, best_loss = min(candidates, key=lambda x: x[1])
    return best_w, best_loss


In [ ]:
# ─── 2) Initialize ───────────────────────────────────────────────────────────
prev_w    = float(input("Start weight (e.g. 0.5):\n") or 0.5)
prev_loss = evaluate_loss(prev_w)
step     = 0.1
tol      = 1e-3    # convergence threshold
max_iter = 20

print(f"\nStarting at weight {prev_w:.3f} with loss {prev_loss:.3f}\n")

# ─── 3) Iterative loop ───────────────────────────────────────────────────────
for iteration in range(1, max_iter + 1):
    # Propose a new weight
    next_w, next_loss = calculate_new_weight(prev_w, prev_loss, step)
    print(f"Iteration {iteration}: suggested weight {next_w:.3f} (prev loss {prev_loss:.3f} → new loss {next_loss:.3f})")

    # 3a) Run your real2real pipeline on next_w
    both_real_main(
        weight_A=next_w,
        input_dir=input_dir,     # reuse same input_dir defined earlier
        output_dir=output_dir,   # reuse same output_dir defined earlier
        num_pairs=1
    )

    # 3b) Prompt user to run SlowFast on the newly generated videos
    print(f"\nNow run SlowFast on videos_real2_{next_w:.3f} and place loss_w{next_w:.2f}.txt in:\n  {loss_dir}")
    input("Press Enter when the new loss file is ready…")

    # 3c) Re-evaluate loss
    new_loss = evaluate_loss(next_w)
    print(f"→ New loss at weight {next_w:.3f}: {new_loss:.3f}\n")

    # 3d) Check for convergence
    if abs(new_loss - prev_loss) < tol:
        print(f"Converged (Δloss {abs(new_loss - prev_loss):.4f} < {tol})")
        break

    # 3e) Prepare for next iteration
    prev_w, prev_loss = next_w, new_loss
    step = step / 2  # Optionally shrink step each round

else:
    print(f"Reached max iterations ({max_iter}) without full convergence.")